<a href="https://colab.research.google.com/github/simonamador/P300-classifier-evaluation/blob/main/lockedin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# P300 analysis on LIS patients
Working with a dataset of two subject, which consists of 4 registers each:


*   2 low accuracy signals
*   2 high accuracy signals


The data aquisition was done with a 256 Hz sample rate. 3 conditions indicated by trigger data:


1.   Distractor (-1)
2.   Nontarget (1)
3.   Target (2)

# Importing dataset, package installation

In [ ]:
!pip install mne
from google.colab import drive
drive.mount('/content/drive')
import mne
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers
from scipy.io import loadmat
from time import sleep

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
route = '/content/drive/MyDrive/BR41N.IO/files'
channels = ['Fz', 'C3', 'Cz', 'C4', 'CP1', 'CPz', 'CP2', 'Pz']
fs = 256
P1_low1 = loadmat(route+'/P1_low1.mat')
P1_low2 = loadmat(route+'/P1_low2.mat')
P1_high1 = loadmat(route+'/P1_high1.mat')
P1_high2 = loadmat(route+'/P1_high2.mat')
P2_low1 = loadmat(route+'/P2_low1.mat')
P2_low2 = loadmat(route+'/P2_low2.mat')
P2_high1 = loadmat(route+'/P2_high1.mat')
P2_high2 = loadmat(route+'/P2_high2.mat')

FileNotFoundError: ignored

# Preprocessing



1.   Convert to Volts
2.   Remove offset
3.   Filtro pasa altas de 1 Hz
4.   Filtro ICA
5.   Segmentación
6.   Normalizacion


In [ ]:
def preprocessing(raw, fs):
  info = mne.create_info(channels,fs,ch_types='eeg')
  infostim = mne.create_info(['STI'], fs, 'stim')

  data = raw['y'].transpose()/10**6
  data = data - np.mean(data)
  n_raw = mne.io.RawArray(data,info)
  n_raw = n_raw.copy().filter(l_freq=1, h_freq=None)

  ica = mne.preprocessing.ICA(n_components=8, random_state=97, max_iter=800)
  ica.fit(n_raw)
  ica.plot_sources(n_raw, show_scrollbars=False)
  sleep(2)
  compex = input('Components to exclude: ')
  compex = compex.split(', ')
  compex_f = [int(n) for n in compex]
  ica.exclude = compex_f

  n_raw.load_data()
  ica.apply(n_raw)
  n_raw.plot(duration=5,show=False)
  plt.show()

  chex = input('Channels to exclude: ')
  chex = chex.split(', ')
  n_raw.drop_channels(chex)
  n_raw.plot(duration=5,show=False)
  plt.show()

  trigger = raw['trig'].transpose()
  for i in range(trigger.size):
    if trigger[0,i] == -1:
      trigger[0,i] = 3
  
  stim_raw = mne.io.RawArray(trigger, infostim)
  n_raw.add_channels([stim_raw], force_update_info=True)

  #-1 = distractor, 1 = non-target, 2=target
  events = mne.find_events(n_raw, stim_channel='STI')
  epochs = mne.Epochs(n_raw, events, tmin=-0.1, tmax=0.6)

  epochs['1'].plot_image(picks='eeg', combine='mean')
  epochs['2'].plot_image(picks='eeg', combine='mean')
  epochs['3'].plot_image(picks='eeg', combine='mean')

  nontarget = epochs['1'].get_data()
  target = epochs['2'].get_data()
  dist = epochs['3'].get_data()
  return nontarget, target, dist

In [ ]:
nt, t, d = preprocessing(P2_high2, fs)

In [ ]:
def minmax(X):
  scaler = sk.preprocessing.MinMaxScaler()
  for i in range(X.shape[0]):
    X[i,:,:] = scaler.fit_transform(X[i,:,:].transpose()).transpose()
  return X

In [ ]:
Xt = t
Xnt = np.concatenate((nt,d))
Xt = minmax(Xt)
Xnt = minmax(Xnt)
yt = np.repeat(1,t.shape[0])
ynt = np.concatenate((np.repeat(0,nt.shape[0]),np.repeat(0,d.shape[0])))


sz = Xt.shape
#Generate matries for training-testing
X_tr = np.empty(shape=(round(sz[0]*1.6),sz[1],sz[2],1))
X_ts = np.empty(shape=(round(sz[0]*1.6),sz[1],sz[2],1))
#Use of permutation to randomly divide the matrix index,
#then divide feature and label matrices into training-testing
#in an 80/20 ratio. 
idx = np.random.permutation(sz[0])
idx_tr = idx[:round(sz[0]*0.8)]
idx_tst, idx_tsnt = idx[round(sz[0]*0.8):], np.random.permutation(int(sz[0]*1.4))
X_tr[:,:,:,0] = np.concatenate((Xt[idx_tr,:,:], Xnt[idx_tr,:,:]))
y_tr = np.concatenate((yt[idx_tr], ynt[idx_tr]))
X_ts[:,:,:,0] = np.concatenate((Xt[idx_tst,:,:], Xnt[idx_tsnt,:,:]))
y_ts = np.concatenate((yt[idx_tst],ynt[idx_tsnt]))

# Procesamiento

LDA Classifier

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

Xtrain = np.zeros((X_tr.shape[0], X_tr.shape[1]*X_tr.shape[2]))
Xtest = np.zeros((X_ts.shape[0], X_ts.shape[1]*X_ts.shape[2]))
for i in range(X_tr.shape[0]):
  v = X_tr[i,:,:,0].flatten()
  Xtrain[i,:] = v

for i in range(X_ts.shape[0]):
  v = X_ts[i,:,:,0].flatten()
  Xtest[i,:] = v


lda = LDA()
X_train = lda.fit(Xtrain, y_tr)
y_test = lda.predict(Xtest)

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=2, random_state=0)

classifier.fit(Xtrain, y_tr)
y_pred = classifier.predict(Xtest)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

cm1 = confusion_matrix(y_ts, y_test)
print(cm1)
print('Accuracy' + str(accuracy_score(y_ts, y_test)))

cm2 = confusion_matrix(y_ts, y_pred)
print(cm2)
print('Accuracy' + str(accuracy_score(y_ts, y_pred)))

MLP classifier

In [ ]:
tf.random.set_seed(42)
MLP = models.Sequential([
    layers.Flatten(),
    layers.Dense(10, activation='linear'),
    layers.Dense(10, activation='linear'),
    layers.Dense(2, activation='softmax')
    ])

MLP.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy'])
history = MLP.fit(X_tr, y_tr, epochs=10, 
                  validation_data=(X_ts, y_ts))
test_loss, test_acc = MLP.evaluate(X_ts,  y_ts, verbose=2)

CNN classifier

In [ ]:
tf.random.set_seed(42)
CNN = models.Sequential([
    layers.Conv2D(3, (2,2), activation='relu', input_shape=(X_tr.shape[1],X_tr.shape[2],1)),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(10, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(2, activation='softmax')
    ])

CNN.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy'])
history = CNN.fit(X_tr, y_tr, epochs=10, 
                  validation_data=(X_ts, y_ts))
test_loss, test_acc = CNN.evaluate(X_ts,  y_ts, verbose=2)